In [1]:
#EMPIRICS - MODEL (Readme)
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Module conducts the following analysis:

# ---- Cost competitiveness forecast and extension to 2100
# ---- Estimation of empirical diffusion model (fitted to natural gas power plants)
# ---- Extended Data Figure 1: Distribution of predictors in the diffusion model
# ---- Supplementary Figure 1: Capacity of natural gas power plants
# ---- Supplementary Figure 2: Spatial autocorrelation
# ---- Supplementary Figure 3: Serial autocorrelation
# ---- Supplementary Figure 6: Cost competititveness trajectory
# ---- Supplementary Table 1: Collinearity test

# Module is input for:

# ---- Empirics - Marginal effects
# ---- Simulation - Model

# Note that this is the first module of the project. Prior to running the working directory and file locations for the data files need to be set depending on where the
# the code is run and where the datafiles have been downloaded to. The datafiles are available in the Github repository under "Data". The powerplants.xls file is based
# on proprietary commercial data from S&P Capital IQ (see data availability note). Licenseholders can download the file from S&P Capital IQ Pro, using the Power Plant Unit
# Screener function and filtering for European Union countries and gen techs.

In [2]:
#SET-UP
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
setwd("/home/h1604190/Data/") #Set Working Directory
Sys.setenv(PROJ_LIB = "/opt/conda/share/proj")
Sys.getenv("PROJ_LIB")

check_and_load <- function(packages) {
  for (pkg in packages) {
    if (!requireNamespace(pkg, quietly = TRUE)) {
      message(paste("Installing missing package:", pkg))
      install.packages(pkg, dependencies = TRUE, repos = "https://cloud.r-project.org")
    }
    if (!(pkg %in% (.packages()))) {
      suppressPackageStartupMessages(library(pkg, character.only = TRUE))
    }
  }
}


required_packages <- c(
  # --- Core data handling ---
  "dplyr",        # Data manipulation (mutate, group_by, summarise, joins, etc.)
  "tidyr",        # Reshaping and cleaning data (pivot_longer, replace_na, etc.)
  "tibble",       # Modern data frame printing and creation (tibble(), as_tibble())

  # --- Spatial analysis and geometry ---
  "sf",           # Spatial features (read/write shapefiles, geometry ops, CRS handling)
  "Matrix",       # Sparse matrices (used for spatial weights matrices)
  "FNN",          # Fast Nearest Neighbour search (KNN-based spatial adjacency)

  # --- Statistical modelling and inference ---
  "geepack",      # Generalized Estimating Equations (for correlated panel data)
  "pscl",         # Pseudo R² and model diagnostics for logistic regression
  "pROC",         # ROC curves and AUC evaluation
  "performance",  # Model quality checks (collinearity, residuals, etc.)
  "corrplot",     # Visualisation of correlation matrices

  # --- Simulation and forecasting utilities ---
  "forecast",     # Time-series and forecasting tools (trend extrapolation)
  "minpack.lm",   # Nonlinear least squares optimisation (Levenberg–Marquardt)

  # --- Visualisation and styling ---
  "ggplot2",      # Core plotting system
  "ggsci",        # Scientific journal colour palettes
  "viridis",      # Perceptually uniform colour scales (for spatial or heat plots)
  "patchwork",    # Combine multiple ggplots into one layout
  "scales",       # Scaling and labelling of axes (rescale, percent, etc.)

  # --- Data I/O ---
  "readxl"        # Import Excel files (.xls / .xlsx)
)
    
check_and_load(required_packages)

[1] "/opt/conda/share/proj"

In [ ]:
#DATAFILES
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

#File paths -> Change file location to where they were downloaded from the repository
inet_pipesegments_geojson_path <- "/home/h1604190/Data1/INET_PipeSegments.geojson"
power_plants_path <- "/home/h1604190/Data1/powerplants.xls"
grid_clean_sf_path <- "/home/h1604190/Data/grid_clean_sf.xlsx"
cost_green_path <- "/home/h1604190/Data/cost_competitiveness_green.xlsx"
cost_fossil_path <- "/home/h1604190/Data/cost_competitiveness_fossil.xlsx"

powerplants <- read_xls(power_plants_path) 
pipe_segments <- st_read(inet_pipesegments_geojson_path)
grid_clean_sf <- st_read(grid_clean_sf_path)
maritime_sf <- grid_clean_sf %>% filter(Industry == "Shipping") #filter maritime port subset
iww_sf <- grid_clean_sf %>% filter(Industry == "Inland Shipping") #filter inland port subset

cost_gap_green <- read_excel(cost_green_path) %>%
  pivot_longer(`2024`:`2050`, names_to = "year", values_to = "green_cost") %>%
  mutate(year = as.integer(year))
cost_gap_fossil <- read_excel(cost_fossil_path) %>%
  pivot_longer(`2024`:`2050`, names_to = "year", values_to = "fossil_cost") %>%
  mutate(year = as.integer(year))

In [ ]:
#INPUTS AND SETTINGS
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Settings
OPTIMIZATION <- FALSE                       # Set to TRUE to run hyperparameter optimization loop. Optimized parameters need to be reinserted manually
currency <- "EUR"                           # Set to EUR or USD
exchange_rate <- 1/1.0321                   # ECB as of 2 Jan 25
noise_multiplier <- 0.5                     # Noise multiplier to avoid perfect classification / overfitting in second-pass calibration equation
k_max_number <- 500                         # Max number of neighbors (spatial weights matrix uses KNN for computational efficiency), set high enough that no neighbors within the distance threshold are cut off
km_cutoff = 120000                          # Distance cutoff for adjacency matrix (based on optimization loop, see above)
carbon_price_setting <- "carbon_price"      # Insert carbon price assumption as "flat_carbon_price", escalating "carbon_price"d
future_years <- 2051:2100                   # Set extension years (data available up to 2050)
analogue <- "Natural Gas"                   # Insert "Wind" for robustness check
start_year <- 1961                          # Start of diffusion in calibration set (1961 for Natural Gas, set 1993 for Wind)

# H2 sector mapping
sector_mapping <- tribble(
  ~sector,                ~green_commodity,       ~fossil_commodity, 
  "Aviation",             "E-Kerosine",           "Kerosine", 
  "Chemicals",            "Green Hydrogen",       "Grey Hydrogen", 
  "Heat",                 "E-Methane",            "Natural Gas", 
  "Heavy duty",           "Green Hydrogen Mobility", "Diesel", 
  "Iron & steel",         "Green Hydrogen Steel", "Natural Gas", 
  "Non-ferrous metals",   "Green Hydrogen",       "Natural Gas",
  "Non-metallic minerals","Green Hydrogen",       "Natural Gas",
  "Other",                "Green Hydrogen",       "Natural Gas",
  "Power",                "E-Methane",            "Natural Gas",
  "Pulp & paper",         "Green Hydrogen",       "Natural Gas", 
  "Refining",             "Green Hydrogen",       "Grey Hydrogen",
  "Shipping",             "E-Methanol",           "Diesel"
)


# LCOE data (Natural Gas, Natural Gas vs Coal from Way et al 2022: Empirically grounded technology forecasts and the energy transition)
lcoe_gap_data <- data.frame(    
  year = 1982:2020,
  
  # Normalized LCOE gap
  lcoe_gap = c(
    0.928349777, 1, 0.815014246, 0.84282832, 0.97121392, 0.857738481, 
    0.84183218, 0.727844509, 0.705788602, 0.654015154, 0.596558097, 
    0.513267971, 0.38288264, 0.344413532, 0.285891647, 0.382830719, 
    0.568830319, 0.586613671, 0.627970952, 0.744666663, 0.549054167, 
    0.684122127, 0.701487841, 0.756124548, 0.768007379, 0.762828347, 
    0.78045041, 0.462188775, 0.281242262, 0.308264951, 0.341901176, 
    0.276427191, 0.229802182, 0.108772523, 0.178188948, 0.143292437, 
    0.120791329, 0, 0.001994048
  ),
  
  # Normalized LCOE
  lcoe = c(
    1, 0.827905548, 0.626635902, 0.645687673, 0.794532244, 0.662973854, 
    0.644532793, 0.512380544, 0.486809904, 0.426786065, 0.360172901, 
    0.263610023, 0.112447054, 0.067847677, 0, 0.11238686, 0.328026546, 
    0.348643775, 0.396591572, 0.531883403, 0.305098952, 0.461690754, 
    0.481823792, 0.545167166, 0.558943594, 0.552939254, 0.573369492, 
    0.698797495, 0.687428492, 0.789765351, 0.70546748, 0.635015203, 
    0.71735831, 0.692006885, 0.697259275, 0.666842932, 0.721612335, 
    0.763884842, 0.871046333
  ),
  
  # Random LCOE gap (for robustness)
  lcoe_gap_random = runif(39, min = 0, max = 0.5),
  
  # Absolute LCOE gap
  lcoe_gap_absolute = c(
    11.3915965, 16.8705048, 2.725119785, 4.851990491, 14.66930727,
    5.992132237, 4.77581823, -3.940525855, -5.627084362, -9.586066826,
    -13.97966029, -20.34864208, -30.31887313, -33.26050689, -37.73552494,
    -30.32284337, -16.09993213, -14.74008498, -11.57760003, -2.654179021,
    -17.61216366, -7.283864017, -5.955952525, -1.778024022, -0.869374463,
    -1.265401695, 0.082112125, -24.25453685, -38.09105146, -36.02469591,
    -33.45262033, -38.45924761, -42.02454219, -51.27936898, -45.97127342,
    -48.63971991, -50.36032181, -59.59692419, -59.44444444
  )
)

# Convert to EUR if specified
if (currency == "EUR") {
  lcoe_gap_data <- lcoe_gap_data %>%
    mutate(across(-year, ~ .x / exchange_rate))
}

# Emissions factors (Odenweller and Ueckerdt 2025, not used in analysis)
ef_diesel <- 0.25 
ef_natural_gas <- 0.265 
ef_kerosine <- 0.32  
ef_grey_h2 <- 0.35
ef_grey_methanol <- 0.37

In [ ]:
#DATA CLEANING AND HANDLING
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# Store initial number of rows
n0 <- nrow(powerplants)

# Ensure numeric columns
powerplants <- powerplants %>%
  mutate(
    LATITUDE        = as.numeric(LATITUDE),
    LONGITUDE       = as.numeric(LONGITUDE),
    YR_UNIT_IN_SVC  = as.numeric(YR_UNIT_IN_SVC)
  )

# Filter to analogue
gasplants_step1 <- powerplants %>%
  filter(PRIMARY_FUEL_TYPE == analogue)   # change to "Wind" for robustness check
n1 <- nrow(gasplants_step1)

# Drop NA
gasplants_step2 <- gasplants_step1 %>% 
  drop_na(LONGITUDE, LATITUDE, YR_UNIT_IN_SVC)
n2 <- nrow(gasplants_step2)


# Exclude plants not yet in service
gasplants_step3 <- gasplants_step2 %>%
  filter(CURRENT_OPER_STATUS %in% c("Operating", "Out of Service"))
n3 <- nrow(gasplants_step3)

# Unique location id + capacity aggregation + dedu
gasplants_step4 <- gasplants_step3 %>%
  mutate(
    location_id = paste(POWER_PLANT, GEN_TECH, YR_UNIT_IN_SVC,
                        LATITUDE, LONGITUDE, sep = "_")
  ) %>%
  group_by(location_id) %>%
  mutate(total_capacity = sum(UNIT_NAMEPLATE_CAPACITY, na.rm = TRUE)) %>%
  ungroup() %>%
  distinct(location_id, .keep_all = TRUE)
n4 <- nrow(gasplants_step4)

# Calculate infrastructure access metrics
gasplants_sf <- gasplants_step4 %>%
  st_as_sf(coords = c("LONGITUDE", "LATITUDE"), crs = 4326)
gas_coords      <- st_coordinates(gasplants_sf)
maritime_coords <- st_coordinates(maritime_sf)
iww_coords      <- st_coordinates(iww_sf)
dist_to_maritime <- apply(
  gas_coords, 1,
  function(pt) min(geosphere::distHaversine(pt, maritime_coords))
)
dist_to_iww <- apply(
  gas_coords, 1,
  function(pt) min(geosphere::distHaversine(pt, iww_coords))
)
pipe_segments_sf <- st_as_sf(pipe_segments)
gasplants_sf <- gasplants_sf %>%
  mutate(
    distance_to_pipeline  = apply(st_distance(gasplants_sf, pipe_segments_sf), 1, min) / 1000,
    distance_to_port      = dist_to_maritime / 1000,
    distance_to_iww       = dist_to_iww / 1000,
    distance_to_waterway  = pmin(distance_to_port, distance_to_iww),
    plant_id              = row_number()
  )
n_before_final_drop <- nrow(gasplants_sf)

# Drop NAs produced by distance calculation
gasplants_sf <- gasplants_sf %>%
  drop_na(distance_to_waterway, distance_to_pipeline)

n_after_final_drop <- nrow(gasplants_sf)

# Expand to panel
years <- seq(
  from = min(gasplants_sf$YR_UNIT_IN_SVC, na.rm = TRUE),
  to   = max(gasplants_sf$YR_UNIT_IN_SVC, na.rm = TRUE)
)

training_data_full <- expand.grid(
  plant_id = gasplants_sf$plant_id,
  year     = years
) %>%
  left_join(gasplants_sf, by = "plant_id") %>%
  mutate(adopted = ifelse(year >= YR_UNIT_IN_SVC, 1, 0)) %>%
  filter(year > start_year)


# Summary of data cleaning steps

cat("\n===== Cleaning Summary =====\n")
cat("Original rows:           ", n0, "\n", sep = "")
cat("After fuel filter:       ", n1, " (", n0 - n1, " removed)\n", sep = "")
cat("After drop_na:           ", n2, " (", n1 - n2, " removed)\n", sep = "")
cat("After status filter:     ", n3, " (", n2 - n3, " removed)\n", sep = "")
cat("After deduplication:     ", n4, " (", n3 - n4, " removed)\n", sep = "")
cat("After distance NA drop:  ", n_after_final_drop, 
    " (", n_before_final_drop - n_after_final_drop, " removed)\n", sep = "")
cat("============================\n\n")

In [ ]:
#COST COMPETITIVENESS FORECAST AND EXTENSION TO 100
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# Forecast functions
wrights_law_forecast <- function(years, values, future_years) {
  C_2050 <- values[years == 2050]
  anchored <- function(t, r) C_2050 * exp(-r * (t - 2050))
  fit <- nlsLM(values ~ anchored(years, r),
               start = list(r = 0.05), lower = 0.001, upper = 1,
               control = nls.lm.control(maxiter = 500))
  r_est <- coef(fit)[["r"]]
  C_2050 * exp(-r_est * (future_years - 2050))
}

arima_forecast <- function(years, values, future_years) {
  if (length(unique(values)) < 5) return(rep(NA, length(future_years)))
  ts_data <- ts(values, start = min(years), frequency = 1)
  forecast(auto.arima(ts_data), h = length(future_years))$mean |> as.numeric()
}

ar1_forecast <- function(years, values, future_years) {
  if (length(unique(values)) < 5) return(rep(NA, length(future_years)))

  ts_data <- ts(values, start = min(years), frequency = 1)

  fit <- arima(
    ts_data,
    order = c(1, 0, 0),
    method = "ML",
    transform.pars = TRUE   # forces |phi| < 1
  )

  preds <- predict(fit, n.ahead = length(future_years))$pred
  as.numeric(preds)
}


# Forecast data
forecast_green <- cost_gap_green %>%
  group_by(commodity, green_scenario) %>%
  group_map(~{
    if (nrow(.x) < 5) return(NULL)
    tibble(
      commodity = .y$commodity,
      green_scenario = .y$green_scenario,
      year = future_years,
      green_cost = wrights_law_forecast(.x$year, .x$green_cost, future_years)
    )
  }) %>% bind_rows()

forecast_fossil <- cost_gap_fossil %>%
  group_by(commodity, fossil_scenario) %>%
  group_map(~{

    if (nrow(.x) < 5) return(NULL)

    is_flat <- (.y$fossil_scenario == "flat_carbon_price")

    tibble(
      commodity       = .y$commodity,
      fossil_scenario = .y$fossil_scenario,
      year            = future_years,
      fossil_cost     = if (is_flat) {
        # AR(1) for flat carbon price
        ar1_forecast(.x$year, .x$fossil_cost, future_years)
      } else {
        # ARIMA for all default carbon price
        arima_forecast(.x$year, .x$fossil_cost, future_years)
      }
    )
  }) %>%
  bind_rows()

# Combine data
green_all  <- bind_rows(cost_gap_green,  forecast_green)
fossil_all <- bind_rows(cost_gap_fossil, forecast_fossil)
scenario_grid <- expand.grid(
  green_scenario = unique(green_all$green_scenario),
  fossil_scenario = unique(fossil_all$fossil_scenario),
  year = sort(unique(c(green_all$year, fossil_all$year)))
)

# Map to sector
sector_data <- sector_mapping %>%
  crossing(scenario_grid) %>%
  left_join(green_all,  by = c("green_commodity" = "commodity", "green_scenario", "year")) %>%
  left_join(fossil_all, by = c("fossil_commodity" = "commodity", "fossil_scenario", "year")) %>%
  filter(!is.na(green_cost), !is.na(fossil_cost)) %>%
  mutate(
    across(c(green_cost, fossil_cost),
           ~ if (currency == "EUR") .x / exchange_rate else .x),
    cost_diff = fossil_cost - green_cost,
    scenario_pair = paste0("green: ", green_scenario, " | fossil: ", fossil_scenario)
  ) %>%
  select(
    sector, year, green_commodity, fossil_commodity,
    green_scenario, fossil_scenario, scenario_pair,
    green_cost, fossil_cost, cost_diff
  )

#Extract cost competitiveness values
cost_gap_data <- sector_data %>%
  filter(fossil_scenario == carbon_price_setting) #Based on carbon price setting in global settings
table(cost_gap_data$sector) #Check -> should be 231 for all sectors
saveRDS(cost_gap_data, "cost_gap_data.rds") #Save for use in simulation modules

In [ ]:
#SUPPLEMENTARY FIGURE 6: COST COMPETITIVENESS TRAJECTORIES
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# ---- Simplified palettes ----
col_green  <- "#1B9E77"   # green-ish for hydrogen
col_fossil <- "#D95F02"   # reddish fossil

lt_default <- "solid"
lt_flat    <- "dotdash"

sector_subtitles <- sector_mapping %>%
  mutate(label = paste0(
    sector, "\n",
    green_commodity, " + ", fossil_commodity
  )) %>%
  select(sector, label) %>%
  deframe()

# ---- Hydrogen ribbon band ----
green_band <- sector_data %>%
  filter(green_scenario %in% c("mean","conservative","progressive")) %>%
  group_by(sector, year) %>%
  summarise(
    green_min = min(green_cost, na.rm = TRUE),
    green_max = max(green_cost, na.rm = TRUE),
    .groups = "drop"
  )

# Mean hydrogen line
green_mean_line <- filter(sector_data, green_scenario == "mean") %>%
  select(sector, year, green_cost)

# ---- Fossil lines ----
fossil_lines <- sector_data %>%
  filter(green_scenario == "mean",
         fossil_scenario != "no_carbon_price") %>%
  mutate(
    tax_type = case_when(
      fossil_scenario == "carbon_price"      ~ "Default carbon price",
      fossil_scenario == "flat_carbon_price" ~ "Flat carbon price",
      TRUE                                   ~ "Default carbon price"
    )
  )

# ---- Plot ----
p_costs <- ggplot() +
  
  # Green ribbon
  geom_ribbon(
    data = green_band,
    aes(x = year, ymin = green_min, ymax = green_max, fill = "Green cost range"),
    alpha = 0.20, color = NA
  ) +
  
  # Green mean line
  geom_line(
    data = green_mean_line,
    aes(x = year, y = green_cost, color = "Green cost"),
    linewidth = 1.6
  ) +
  
  # Fossil cost
  geom_line(
    data = fossil_lines,
    aes(x = year, y = fossil_cost,
        linetype = tax_type,
        color = "Fossil cost"),
    linewidth = 1.3
  ) +
  
  facet_wrap(
    ~sector, ncol = 4, scales = "fixed",
    labeller = labeller(sector = sector_subtitles)
  ) +
  
  # ---- Legends ----
  scale_fill_manual(
    name = "",
    values = c("Green cost range" = col_green)
  ) +
  scale_color_manual(
    name = "",
    values = c("Green cost" = col_green,
               "Fossil cost" = col_fossil)
  ) +
  
  scale_linetype_manual(
    name = "Carbon tax regime",
    values = c("Default tax" = lt_default,
               "Flat carbon price"    = lt_flat)
  ) +
  
  scale_x_continuous(sec.axis = dup_axis(name = NULL, labels = NULL)) +
  scale_y_continuous(sec.axis = dup_axis(name = NULL, labels = NULL)) +
  
  labs(
    title = "Hydrogen (and derivative) and fossil cost trajectories",
    x     = "Year",
    y     = "EUR/MWh"
  ) +
  geom_hline(yintercept = 0, color = "black", linewidth = 0.75) +
  
  theme_minimal(base_size = 18) +
  theme(
    panel.grid.major = element_blank(),
    panel.grid.minor = element_blank(),
    axis.line       = element_line(color = "black"),
    axis.ticks      = element_line(color = "black"),
    axis.text       = element_text(size = 16),
    axis.title      = element_text(size = 18),
    plot.title      = element_text(size = 20, hjust = 0.5),
    
    strip.text      = element_text(size = 16),
    
    # ---- Unified horizontal legend ----
    legend.position   = "bottom",
    legend.box        = "horizontal",
    legend.direction  = "horizontal",
    legend.title      = element_text(size = 16),
    legend.text       = element_text(size = 16),
    legend.spacing.x  = unit(0.6, "cm"),
    legend.key.width  = unit(1.8, "cm"),
    legend.key.height = unit(0.8, "cm"),
    
    axis.text.x.top    = element_blank(),
    axis.text.y.right  = element_blank(),
    axis.ticks.x.top   = element_blank(),
    axis.ticks.y.right = element_blank()
  )

options(repr.plot.width = 20, repr.plot.height = 15, repr.plot.res = 800)
p_costs


ggsave(
  "si-cost.pdf", p_costs, device = pdf,
  width = 20, height = 15, units = "in", dpi = 800
)

In [ ]:
#OPTIMIZATION MODULE: DISTANCE CUT-OFF AND RANDOM NOISE PARAMETER
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

#Note that in order to run this module, OPTIMIZATION needs to be set to YES in Global Settings, and the outcome values need to be re-inserted manually into settings

if (OPTIMIZATION) {

  # Hyperparameter grid
  distance_cutoffs <- seq(10000, 250000, by = 10000)
  noise_multipliers <- c(0.1, 0.25, 0.5, 0.75, 1)
  grid <- expand.grid(distance_cutoff = distance_cutoffs,
                      noise_multiplier = noise_multipliers)
  grid$mse <- NA_real_
  grid$auc <- NA_real_

  # CRS settings
  crs_proj <- 3035
  gasplants_sf <- st_transform(gasplants_sf, crs = crs_proj)
  plant_ids <- gasplants_sf$plant_id

  # Spatial weights function
  compute_distance_weights <- function(gasplants_sf, distance_cutoff, projected_crs = crs_proj) {
    if (st_is_longlat(gasplants_sf)) {
      gasplants_proj <- st_transform(gasplants_sf, crs = projected_crs)
    } else {
      gasplants_proj <- gasplants_sf
    }

    coords <- st_coordinates(gasplants_proj)
    dmat <- as.matrix(dist(coords))
    neighbor_mask <- dmat <= distance_cutoff & dmat > 0

    i <- rep(1:nrow(dmat), times = ncol(dmat))
    j <- rep(1:ncol(dmat), each = nrow(dmat))
    valid <- which(as.vector(neighbor_mask))
    i <- i[valid]; j <- j[valid]
    x <- rep(1, length(valid))

    neighbors_matrix <- sparseMatrix(i = i, j = j, x = x,
                                     dims = c(nrow(dmat), ncol(dmat)))
    spatial_weights <- neighbors_matrix / pmax(rowSums(neighbors_matrix), 1)
    return(spatial_weights)
  }

  # Grid search
  for (i in seq_len(nrow(grid))) {
    cutoff <- grid$distance_cutoff[i]
    noise_multiplier <- grid$noise_multiplier[i]

    cat("Evaluating:", cutoff, "m,", noise_multiplier, "noise\n")

    # Compute spatial weights
    spatial_weights <- compute_distance_weights(gasplants_sf, cutoff)
    if (!inherits(spatial_weights, "dgCMatrix")) {
      warning("Invalid weight matrix – skipping.")
      next
    }

    # Prepare data
    training_data_full <- training_data_full %>%
      filter(plant_id %in% plant_ids)

    training_data_full$spatial_influence <- NA_real_
    for (y in unique(training_data_full$year)) {
      this_year <- training_data_full$year == y
      adopted_vec <- training_data_full$adopted[this_year]
      ordered_ids <- training_data_full$plant_id[this_year]

      idx <- match(plant_ids, ordered_ids)
      if (any(is.na(idx))) next
      adopted_ordered <- adopted_vec[idx]
      spatial_influence_year <- as.numeric(spatial_weights %*% adopted_ordered)

      training_data_full$spatial_influence[this_year][idx] <- spatial_influence_year
    }

    calibration_data <- training_data_full %>%
      group_by(year) %>%
      mutate(trend = mean(spatial_influence, na.rm = TRUE)) %>%
      ungroup() %>%
      mutate(spatial_influence_detrended = spatial_influence - trend) %>%
      arrange(plant_id, year) %>%
      group_by(plant_id) %>%
      mutate(adopted_lag = lag(adopted)) %>%
      ungroup() %>%
      filter(!is.na(adopted), !is.na(adopted_lag),
             !is.na(spatial_influence_detrended),
             !is.na(distance_to_waterway), !is.na(distance_to_pipeline)) %>%
      mutate(sector = "Calibration")

    # Train Validation Split
    set.seed(123)
    split_index <- sample(seq_len(nrow(calibration_data)), size = 2/3 * nrow(calibration_data))
    train_data <- calibration_data[split_index, ]
    validation_data <- calibration_data[-split_index, ]

    train_data <- train_data %>%
      arrange(plant_id, year) %>%
      mutate(plant_id = as.factor(plant_id), year = as.numeric(year))
    validation_data <- validation_data %>%
      arrange(plant_id, year) %>%
      mutate(plant_id = as.factor(plant_id), year = as.numeric(year))

    # First-pass GEE
    first_pass_train <- geeglm(
      adopted ~ spatial_influence_detrended + distance_to_waterway + distance_to_pipeline,
      id = plant_id, data = train_data, family = binomial, corstr = "ar1"
    )
    train_data$cost_proxy <- residuals(first_pass_train, type = "response")

    first_pass_validation <- geeglm(
      adopted ~ spatial_influence_detrended + distance_to_waterway + distance_to_pipeline,
      id = plant_id, data = validation_data, family = binomial, corstr = "ar1"
    )
    validation_data$cost_proxy <- residuals(first_pass_validation, type = "response")

    # Add random noise
    noise_sd <- sd(train_data$cost_proxy, na.rm = TRUE) * noise_multiplier
    train_data$cost_proxy_noise <- train_data$cost_proxy + rnorm(nrow(train_data), 0, noise_sd)
    validation_data$cost_proxy_noise <- validation_data$cost_proxy + rnorm(nrow(validation_data), 0, noise_sd)

    # Rescale to simulation range
    sim_min <- min(cost_gap_data$cost_diff, na.rm = TRUE)
    sim_max <- max(cost_gap_data$cost_diff, na.rm = TRUE)
    sim_zero_closest <- cost_gap_data$cost_diff[which.min(abs(cost_gap_data$cost_diff))]

    calib_min <- min(train_data$cost_proxy_noise, na.rm = TRUE)
    calib_max <- max(train_data$cost_proxy_noise, na.rm = TRUE)
    calib_zero_closest <- train_data$cost_proxy_noise[which.min(abs(train_data$cost_proxy_noise))]

    scaling_factor_below_zero <- (sim_zero_closest - sim_min) / (calib_zero_closest - calib_min)
    scaling_factor_above_zero <- (sim_max - sim_zero_closest) / (calib_max - calib_zero_closest)

    train_data <- train_data %>%
      mutate(cost_proxy_scaled = case_when(
        cost_proxy_noise < calib_zero_closest ~ sim_min + scaling_factor_below_zero * (cost_proxy_noise - calib_min),
        cost_proxy_noise > calib_zero_closest ~ sim_zero_closest + scaling_factor_above_zero * (cost_proxy_noise - calib_zero_closest),
        TRUE ~ sim_zero_closest
      )) %>%
      filter(!is.na(cost_proxy_scaled))

    valid_min <- min(validation_data$cost_proxy_noise, na.rm = TRUE)
    valid_max <- max(validation_data$cost_proxy_noise, na.rm = TRUE)
    valid_zero_closest <- validation_data$cost_proxy_noise[which.min(abs(validation_data$cost_proxy_noise))]

    scaling_factor_below_zero <- (sim_zero_closest - sim_min) / (valid_zero_closest - valid_min)
    scaling_factor_above_zero <- (sim_max - sim_zero_closest) / (valid_max - valid_zero_closest)

    validation_data <- validation_data %>%
      mutate(cost_proxy_scaled = case_when(
        cost_proxy_noise < valid_zero_closest ~ sim_min + scaling_factor_below_zero * (cost_proxy_noise - valid_min),
        cost_proxy_noise > valid_zero_closest ~ sim_zero_closest + scaling_factor_above_zero * (cost_proxy_noise - valid_zero_closest),
        TRUE ~ sim_zero_closest
      )) %>%
      filter(!is.na(cost_proxy_scaled))

    # Replace with observed LCOE in validation
    validation_data <- validation_data %>%
      left_join(lcoe_gap_data %>% mutate(lcoe_gap = -lcoe_gap_absolute), by = "year") %>%
      filter(!is.na(lcoe_gap)) %>%
      mutate(cost_proxy_scaled = lcoe_gap)

    # Force numeric
    force_numeric <- function(df) {
      df %>% mutate(across(c(cost_proxy_scaled, spatial_influence_detrended,
                             distance_to_waterway, distance_to_pipeline), as.numeric))
    }
    train_data <- force_numeric(train_data)
    validation_data <- force_numeric(validation_data)

    # Second-pass GEE
    second_pass <- geeglm(
      adopted ~ cost_proxy_scaled * spatial_influence_detrended + distance_to_waterway + distance_to_pipeline, id = plant_id, data = train_data,
      family = binomial, corstr = "ar1" 
    )

    validation_data$predicted_adoption <- predict(second_pass, newdata = validation_data, type = "response")

    # Metrics
    grid$mse[i] <- mean((validation_data$adopted - validation_data$predicted_adoption)^2, na.rm = TRUE)
    roc_obj <- pROC::roc(response = validation_data$adopted,
                         predictor = validation_data$predicted_adoption)
    grid$auc[i] <- as.numeric(pROC::auc(roc_obj))
  }

  # Plot results
  print(
    ggplot(grid, aes(x = factor(distance_cutoff), y = factor(noise_multiplier), fill = auc)) +
      geom_tile() +
      geom_text(aes(label = round(auc, 3)), size = 3) +
      scale_fill_gradient(low = "red", high = "green") +
      labs(
        title = "Validation AUC across hyperparameters",
        x = "Distance Cutoff (m)", y = "Noise Multiplier"
      ) +
      theme_minimal()
  )

  # Report best
  best_idx <- which.max(grid$auc)
  cat("Best distance_cutoff:", grid$distance_cutoff[best_idx],
      "noise_multiplier:", grid$noise_multiplier[best_idx],
      "with AUC =", round(grid$auc[best_idx], 4), "\n")

  saveRDS(grid, "grid_search_results.rds")
}

In [ ]:
#EMPIRICAL MODEL
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# CRS transformation 

crs_proj <- 3035
gasplants_sf <- st_transform(gasplants_sf, crs = crs_proj)

# Spatial weights function

compute_distance_weights <- function(gasplants_sf, distance_cutoff = km_cutoff, projected_crs = crs_proj) {
  if (st_is_longlat(gasplants_sf)) {
    message("Transforming to projected CRS (EPSG:", projected_crs, ") for distance computation.")
    gasplants_proj <- st_transform(gasplants_sf, crs = projected_crs)
  } else {
    gasplants_proj <- gasplants_sf
  }

  coords <- st_coordinates(gasplants_proj)
  dmat <- as.matrix(dist(coords))

  neighbor_mask <- dmat <= distance_cutoff & dmat > 0

  i <- rep(1:nrow(dmat), times = ncol(dmat))
  j <- rep(1:ncol(dmat), each = nrow(dmat))
  valid <- which(as.vector(neighbor_mask))

  i <- i[valid]
  j <- j[valid]
  x <- rep(1, length(valid))

  neighbors_matrix <- sparseMatrix(i = i, j = j, x = x,
                                   dims = c(nrow(dmat), ncol(dmat)))

  spatial_weights <- neighbors_matrix / pmax(rowSums(neighbors_matrix), 1)
  return(spatial_weights)
}

spatial_weights <- compute_distance_weights(gasplants_sf)

# Prepare calibration data

        # Ensure plant_id order matches between gasplants_sf and training_data_full
        plant_ids <- gasplants_sf$plant_id
        training_data_full <- training_data_full %>% filter(plant_id %in% plant_ids)
        
        # Pre-allocate spatial influence (spatial spilllover) column
        training_data_full$spatial_influence <- NA_real_
        
        # Compute spatial influence year-by-year
        for (y in unique(training_data_full$year)) {
          this_year <- training_data_full$year == y
          adopted_vec <- training_data_full$adopted[this_year]
          ordered_ids <- training_data_full$plant_id[this_year]
          
          idx <- match(plant_ids, ordered_ids)
          if (any(is.na(idx))) next  # skip years with missing data
          
          adopted_ordered <- adopted_vec[idx]
          spatial_influence_year <- as.numeric(spatial_weights %*% adopted_ordered)
          
          training_data_full$spatial_influence[this_year][idx] <- spatial_influence_year
        }
        
        # Detrend spatial influence to avoid temporal trends getting incorporated in the coefficient, continue data prep
        calibration_data <- training_data_full %>%
          group_by(year) %>%
          mutate(trend = mean(spatial_influence, na.rm = TRUE)) %>%
          ungroup() %>% 
          mutate(spatial_influence_detrended = spatial_influence - trend) %>%
          arrange(plant_id, year) %>%
          group_by(plant_id) %>%
          mutate(adopted_lag = lag(adopted)) %>%
          ungroup() %>%
          filter(!is.na(adopted),!is.na(adopted_lag),!is.na(spatial_influence_detrended),!is.na(distance_to_waterway),!is.na(distance_to_pipeline)) %>%
          mutate(sector = "Calibration") #dummy
        
        # Train / validation split
        set.seed(123)
        split_index <- sample(seq_len(nrow(calibration_data)), size = 2/3 * nrow(calibration_data))
        train_data <- calibration_data[split_index, ]
        validation_data <- calibration_data[-split_index, ]
        
        train_data <- train_data %>%
          arrange(plant_id, year) %>%
          mutate(
            plant_id = as.factor(plant_id),
            year = as.numeric(year)
          )
        
        validation_data <- validation_data %>%
          arrange(plant_id, year) %>%
          mutate(
            plant_id = as.factor(plant_id),
            year = as.numeric(year)
          )

# Estimate first-pass model (compute cost competitiveness proxy)

        # First-pass model
        first_pass_train <- geeglm(
          adopted ~ spatial_influence_detrended  + distance_to_waterway + distance_to_pipeline,
          id = plant_id,
          data = train_data,
          family = binomial,
          corstr = "ar1"
        )
        train_data$cost_proxy <- residuals(first_pass_train, type = "response")
        summary(first_pass_train)
        
        first_pass_validation <- geeglm(
          adopted ~ spatial_influence_detrended + distance_to_waterway + distance_to_pipeline,
          id = plant_id,
          data = validation_data,
          family = binomial,
          corstr = "ar1"
        )
        validation_data$cost_proxy <- residuals(first_pass_validation, type = "response")
        
        # Add random noise to avoid overfitting / perfect classification in second pass
        noise_sd <- sd(train_data$cost_proxy) * noise_multiplier
        train_data$cost_proxy_noise <- train_data$cost_proxy + rnorm(nrow(train_data), 0, noise_sd)
        validation_data$cost_proxy_noise <- validation_data$cost_proxy + rnorm(nrow(validation_data), 0, noise_sd)
        
        # Re-scale cost proxy to be consistent with simulation environment (h2 cost)
        cost_gap_data_central <- cost_gap_data %>% filter(green_scenario == "mean" & fossil_scenario == carbon_price_setting)
        sim_min <- min(cost_gap_data$cost_diff, na.rm = TRUE)
        sim_max <- max(cost_gap_data$cost_diff, na.rm = TRUE)
        sim_zero_closest <- cost_gap_data$cost_diff[which.min(abs(cost_gap_data$cost_diff))]
        
        calib_min <- min(train_data$cost_proxy_noise, na.rm = TRUE)
        calib_max <- max(train_data$cost_proxy_noise, na.rm = TRUE)
        calib_zero_closest <- train_data$cost_proxy_noise[which.min(abs(train_data$cost_proxy_noise))]
        
        scaling_factor_below_zero <- (sim_zero_closest - sim_min) / (calib_zero_closest - calib_min)
        scaling_factor_above_zero <- (sim_max - sim_zero_closest) / (calib_max - calib_zero_closest)
        train_data <- train_data %>%
          mutate(
            cost_proxy_scaled = case_when(
              cost_proxy_noise < calib_zero_closest ~ sim_min + scaling_factor_below_zero * (cost_proxy_noise - calib_min),
              cost_proxy_noise > calib_zero_closest ~ sim_zero_closest + scaling_factor_above_zero * (cost_proxy_noise - calib_zero_closest),
              TRUE ~ sim_zero_closest
            )
          ) 
        train_data <- train_data %>% filter(!is.na(cost_proxy_scaled)) #drop missing values if any
        
        valid_min <- min(validation_data$cost_proxy_noise, na.rm = TRUE)
        valid_max <- max(validation_data$cost_proxy_noise, na.rm = TRUE)
        valid_zero_closest <- validation_data$cost_proxy_noise[which.min(abs(validation_data$cost_proxy_noise))]
        
        # Scaling factors
        scaling_factor_below_zero <- (sim_zero_closest - sim_min) / (valid_zero_closest - valid_min)
        scaling_factor_above_zero <- (sim_max - sim_zero_closest) / (valid_max - valid_zero_closest)
        validation_data <- validation_data %>%
          mutate(
            cost_proxy_scaled = case_when(
              cost_proxy_noise < valid_zero_closest ~ sim_min + scaling_factor_below_zero * (cost_proxy_noise - valid_min),
              cost_proxy_noise > valid_zero_closest ~ sim_zero_closest + scaling_factor_above_zero * (cost_proxy_noise - valid_zero_closest),
              TRUE ~ sim_zero_closest
            )
          ) 
        validation_data <- validation_data %>% filter(!is.na(cost_proxy_scaled))
        
        # Check cost competitiveness proxy vs lcoe data (natural gas vs coal)
        validation_data <- validation_data %>%
          left_join(lcoe_gap_data %>% mutate(lcoe_gap = -lcoe_gap_absolute), by = "year")
        train_data <- train_data %>%
          left_join(lcoe_gap_data %>% mutate(lcoe_gap = -lcoe_gap_absolute), by = "year")
        print(paste("Correlation:", round(cor(validation_data$cost_proxy_scaled, validation_data$lcoe_gap, use = "complete.obs"), 3)))
        print(paste("Correlation:", round(cor(train_data$cost_proxy_scaled, train_data$spatial_influence_detrended, use = "complete.obs"), 3)))
        
        validation_data <- validation_data %>%
        group_by(year) %>%
        mutate(cost_proxy_avg = mean(cost_proxy_scaled)) %>%
        ungroup()
        
        # Replace cost proxy with observed lcoe data for validation
        validation_data$cost_proxy_scaled <- validation_data$lcoe_gap

        # keep only rows with LCOE data
        validation_data <- validation_data %>%
        filter(!is.na(cost_proxy_scaled))

# Estimate second-pass model (assign coefficients)

        # Force numeric
        force_numeric <- function(df) {
          df %>%
            mutate(
              across(
                c(cost_proxy_scaled, spatial_influence_detrended,
                  distance_to_waterway, distance_to_pipeline),
                as.numeric
              )
            )
        }
        
        train_data      <- force_numeric(train_data)
        validation_data <- force_numeric(validation_data)
        
        
        # Second pass model
        second_pass <- geeglm(
          adopted ~ cost_proxy_scaled * spatial_influence_detrended +
                    distance_to_waterway + distance_to_pipeline,
          id      = plant_id,
          data    = train_data,
          family  = binomial,
          corstr  = "ar1"
        )
        summary(second_pass)

#Model evaluation

        #Predict on train and validation sets
        train_data$predicted_adoption <- predict(second_pass, newdata = train_data, type = "response")
        validation_data$predicted_adoption <- predict(second_pass, newdata = validation_data, type = "response")

        #Compute ROC objects
        roc_train <- roc(response = train_data$adopted, predictor = train_data$predicted_adoption)
        roc_val   <- roc(response = validation_data$adopted, predictor = validation_data$predicted_adoption)
        
        #AUCs
        cat("AUC Train:", round(auc(roc_train), 3), "\n")
        cat("AUC Validation:", round(auc(roc_val), 3), "\n")
        
        roc_train_df <- data.frame(
          fpr = 1 - roc_train$specificities,
          tpr = roc_train$sensitivities,
          dataset = "Train"
        )
        roc_val_df <- data.frame(
          fpr = 1 - roc_val$specificities,
          tpr = roc_val$sensitivities,
          dataset = "Validation"
        )
        roc_df <- rbind(roc_train_df, roc_val_df)
        
        #AUC plot
        nrc_colors <- pal_npg("nrc")(10)  
        
        p_roc <- ggplot(roc_df, aes(x = fpr, y = tpr, color = dataset)) +
          geom_line(linewidth = 1.2) +
          geom_abline(slope = 1, intercept = 0, linetype = "dashed", color = "grey50") +
          scale_color_manual(values = nrc_colors[1:2], name = "Dataset") +  
          scale_x_continuous(sec.axis = dup_axis(name = NULL, labels = NULL)) +
          scale_y_continuous(sec.axis = dup_axis(name = NULL, labels = NULL)) +
          labs(
            title = "ROC curves: Train vs Validation",
            x = "False positive rate",
            y = "True positive rate"
          ) +
          theme_minimal(base_size = 14) +
          theme(
            axis.line = element_line(color = "black"),
            axis.ticks = element_line(color = "black"),
            axis.text = element_text(size = 14),
            axis.title = element_text(size = 14),
            plot.title = element_text(size = 14, face = "plain", hjust = 0.5),
            legend.position = "right"
          )  
        
        #MSE
        mse_train <- mean((train_data$adopted - train_data$predicted_adoption)^2)
        mse_val   <- mean((validation_data$adopted - validation_data$predicted_adoption)^2, na.rm = TRUE)
        cat("MSE Train:", mse_train, "\n")
        cat("MSE Validation:", mse_val, "\n")
        
        options(repr.plot.width = 22, repr.plot.height = 8, repr.plot.res = 800)
        p_roc

#Save data and model
saveRDS(train_data, file = "train_data.rds")
saveRDS(validation_data, file = "validation_data.rds")
saveRDS(second_pass, file = "second_pass.rds")

In [ ]:
#EXTENDED DATA FIGURE 1: DISTRIBUTION OF PREDICTORS
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
#Create a long-format dataframe 
train_data_long <- train_data %>%
  select(cost_proxy_scaled, spatial_influence, spatial_influence_detrended, distance_to_waterway, distance_to_pipeline) %>%
  pivot_longer(cols = everything(), names_to = "variable", values_to = "value")

#Named vector for facet titles
variable_labels <- c(
  cost_proxy_scaled       = "Cost competitiveness, EUR/MWh",
  spatial_influence_detrended = "Spatial spillover, Index, 1- to 1",
  distance_to_waterway      = "Distance to navigable port (km)",
  distance_to_pipeline      = "Distance to pipeline corridor (km)"
)

#Plot
nrc_colors <- pal_npg("nrc")(10)

p_predictors <- ggplot(train_data_long, aes(x = value)) +
  geom_histogram(bins = 30, fill = nrc_colors[2], colour = "white") +
  facet_wrap(~variable, scales = "free",
             labeller = labeller(variable = variable_labels)) +
  geom_vline(xintercept = 0, linetype = "dashed", colour = "black") +
  scale_x_continuous(sec.axis = dup_axis(name = NULL, labels = NULL)) +
  scale_y_continuous(sec.axis = dup_axis(name = NULL, labels = NULL)) +
  labs(
    title = "Distribution of predictor variables",
    x = "Value", y = "Count"
  ) +
  theme_minimal(base_size = 14) +
  theme(
    axis.line   = element_line(color = "black"),
    axis.ticks  = element_line(color = "black"),
    axis.text   = element_text(size = 16),
    axis.title  = element_text(size = 16),
    plot.title  = element_text(size = 18, face = "plain", hjust = 0.5),
    strip.text  = element_text(size = 16),
    axis.text.x.top = element_blank(),
    axis.text.y.right = element_blank(),
    axis.ticks.x.top = element_blank(),
    axis.ticks.y.right = element_blank()
  )

options(repr.plot.width = 14, repr.plot.height = 8, repr.plot.res = 800)
p_predictors
ggsave("predictor_distributions.pdf", p_predictors,
       device = cairo_pdf, width = 14, height = 8, units = "in", dpi = 800)

In [ ]:
#SUPPLEMENTARY FIGURE 1: CAPCITY OF NATURAL GAS POWER PLANTS
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
nrc_colors <- pal_npg("nrc")(10)

p_capacity <- training_data_full %>%
  group_by(year) %>%
  summarise(
    capacity   = sum(adopted * UNIT_NAMEPLATE_CAPACITY, na.rm = TRUE),  
    n_adopted  = sum(adopted, na.rm = TRUE),
    n_total    = n(),
    adoption_rate = n_adopted / n_total,
    .groups = "drop"
  ) %>%
  ggplot(aes(x = year, y = capacity)) +
  geom_line(color = nrc_colors[2], linewidth = 1.2) +
  geom_point(color = nrc_colors[2], size = 2.5) +
  scale_x_continuous(sec.axis = dup_axis(name = NULL, labels = NULL)) +
  scale_y_continuous(sec.axis = dup_axis(name = NULL, labels = NULL)) +
  labs(
    title = "Gas plant capacity over time",
    y = "Capacity of adopted plants (MW)",
    x = "Year"
  ) +
  theme_minimal(base_size = 18) +
  theme(
    panel.grid.major = element_blank(),
    panel.grid.minor = element_blank(),
    axis.line   = element_line(color = "black"),
    axis.ticks  = element_line(color = "black"),
    axis.text   = element_text(size = 18),
    axis.title  = element_text(size = 18),
    plot.title  = element_text(size = 18, face = "plain", hjust = 0.5),
    legend.position = "none"
  )

options(repr.plot.width = 12, repr.plot.height = 6, repr.plot.res = 800)
p_capacity
ggsave("si-gas-capacity.pdf", p_capacity,
       device = cairo_pdf, width = 12, height = 6, units = "in", dpi = 400)

adoption_summary <- training_data_full %>%
  group_by(year) %>%
  summarise(
    capacity       = sum(adopted * UNIT_NAMEPLATE_CAPACITY, na.rm = TRUE),
    n_adopted      = sum(adopted, na.rm = TRUE),
    n_total        = n(),
    adoption_rate  = n_adopted / n_total,
    .groups = "drop"
  )

# find first year where adoption exceeds 1%
year_reaches_1pct <- adoption_summary %>%
  filter(adoption_rate >= 0.01) %>%
  slice_head(n = 1)

year_reaches_1pct


In [ ]:
#SUPPLEMENTARY FIGURE 2: SPATIAL AUTOCORRELATION
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
#Extract residuals from both AR(1) models
resid_first_pass <- residuals(first_pass_train, type = "response")
resid_second_pass <- residuals(second_pass, type = "response")

#Ensure correct ordering: gasplants_sf plant_id must match train_data$plant_id
plant_id_order <- match(train_data$plant_id, gasplants_sf$plant_id)
if (any(is.na(plant_id_order))) stop("Mismatch in plant_id between train_data and gasplants_sf")

#Subset the spatial_weight matrix to only those rows in train_data
W_subset <- spatial_weights[plant_id_order, plant_id_order]

#Compute spatial lag of residuals
lag_resid_first_pass <- as.numeric(W_subset %*% resid_first_pass)
lag_resid_second_pass <- as.numeric(W_subset %*% resid_second_pass)

moran_df <- data.frame(
  resid_first_pass = resid_first_pass,
  lag_resid_first_pass = lag_resid_first_pass,
  resid_second_pass = resid_second_pass,
  lag_resid_second_pass = lag_resid_second_pass
)

#Plot
nrc_colors <- pal_npg("nrc")(10)
p1 <- ggplot(moran_df, aes(resid_first_pass, lag_resid_first_pass)) +
  geom_point(alpha = 0.5, color = nrc_colors[2]) +
  geom_smooth(method = "lm", se = FALSE, colour = "black", linewidth = 0.8) +
  scale_x_continuous(sec.axis = dup_axis(name = NULL, labels = NULL)) +
  scale_y_continuous(sec.axis = dup_axis(name = NULL, labels = NULL)) +
  labs(
    title = "Moran plot: First-pass",
    x = "Residual",
    y = "Spatial lag of residual"
  ) +
  theme_minimal(base_size = 20) +
  theme(
    panel.grid.major = element_blank(),
    panel.grid.minor = element_blank(),
    axis.line   = element_line(color = "black"),
    axis.ticks  = element_line(color = "black"),
    axis.text   = element_text(size = 18),
    axis.title  = element_text(size = 18),
    plot.title  = element_text(size = 18, face = "plain", hjust = 0.5),
    axis.text.x.top = element_blank(),
    axis.text.y.right = element_blank(),
    axis.ticks.x.top = element_blank(),
    axis.ticks.y.right = element_blank()
  )

p2 <- ggplot(moran_df, aes(resid_second_pass, lag_resid_second_pass)) +
  geom_point(alpha = 0.5, color = nrc_colors[4]) +
  geom_smooth(method = "lm", se = FALSE, colour = "black", linewidth = 0.8) +
  scale_x_continuous(sec.axis = dup_axis(name = NULL, labels = NULL)) +
  scale_y_continuous(sec.axis = dup_axis(name = NULL, labels = NULL)) +
  labs(
    title = "Moran plot: Second pass",
    x = "Residual",
    y = "Spatial lag of residual"
  ) +
  theme_minimal(base_size = 20) +
  theme(
    panel.grid.major = element_blank(),
    panel.grid.minor = element_blank(),
    axis.line   = element_line(color = "black"),
    axis.ticks  = element_line(color = "black"),
    axis.text   = element_text(size = 18),
    axis.title  = element_text(size = 18),
    plot.title  = element_text(size = 18, face = "plain", hjust = 0.5),
    axis.text.x.top = element_blank(),
    axis.text.y.right = element_blank(),
    axis.ticks.x.top = element_blank(),
    axis.ticks.y.right = element_blank()
  )


final_moran <- p1 + p2 +
  plot_annotation(tag_levels = "a",
                  theme = theme(plot.tag = element_text(face = "bold", size = 18),
                                plot.tag.position = c(0, 1)))

options(repr.plot.width = 20, repr.plot.height = 10, repr.plot.res = 800)
final_moran
ggsave("moran_scatterplots.pdf", final_moran,
       device = cairo_pdf, width = 12, height = 6, units = "in", dpi = 300)

In [ ]:
#SUPPLEMENTARY FIGURE 3: SERIAL AUTOCORRELATION
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------nrc_colors <- pal_npg("nrc")(10)

#ACF dataframe
get_acf_df <- function(resid_vec, years) {
  resid_time_series <- tapply(resid_vec, years, mean, na.rm = TRUE)
  acf_obj <- acf(resid_time_series, plot = FALSE)
  with(acf_obj, data.frame(lag, acf))
}

acf_df_first_pass  <- get_acf_df(resid_first_pass,  train_data$year)
acf_df_second_pass <- get_acf_df(resid_second_pass, train_data$year)

#Common y-limits + confidence band
combined_acf_vals <- c(acf_df_first_pass$acf, acf_df_second_pass$acf)
ylim_range <- range(combined_acf_vals, na.rm = TRUE)

n          <- length(unique(train_data$year))
conf_level <- qnorm((1 + 0.95)/2) / sqrt(n)

#Plot
make_acf_plot_fixed <- function(acf_df, model_title, color_line, ylim_range, conf_level) {
  ggplot(acf_df, aes(x = lag, y = acf)) +
    geom_segment(aes(xend = lag, y = 0, yend = acf), color = color_line) +
    geom_hline(yintercept = 0, color = "black") +
    geom_hline(yintercept = c(-conf_level, conf_level),
               linetype = "dashed", color = "grey50") +
    coord_cartesian(ylim = ylim_range) +
    scale_x_continuous(sec.axis = dup_axis(name = NULL, labels = NULL)) +
    scale_y_continuous(sec.axis = dup_axis(name = NULL, labels = NULL)) +
    labs(
      title = model_title,
      x = "Lag",
      y = "ACF"
    ) +
    theme_minimal(base_size = 20) +
    theme(
      panel.grid.major = element_blank(),
      panel.grid.minor = element_blank(),
      axis.line   = element_line(color = "black"),
      axis.ticks  = element_line(color = "black"),
      axis.text   = element_text(size = 18),
      axis.title  = element_text(size = 18),
      plot.title  = element_text(size = 18, face = "plain", hjust = 0.5),
      axis.text.x.top    = element_blank(),
      axis.text.y.right  = element_blank(),
      axis.ticks.x.top   = element_blank(),
      axis.ticks.y.right = element_blank()
    )
}

acf_first_pass <- make_acf_plot_fixed(
  acf_df_first_pass,
  "ACF of mean residuals: First-pass (logit model)",
  nrc_colors[2],
  ylim_range,
  conf_level
)

acf_second_pass <- make_acf_plot_fixed(
  acf_df_second_pass,
  "ACF of mean residuals: Second-pass (cost model)",
  nrc_colors[4],
  ylim_range,
  conf_level
)

final_acf <- acf_first_pass + acf_second_pass +
  plot_annotation(
    tag_levels = "a",
    theme = theme(
      plot.tag = element_text(face = "bold", size = 18),
      plot.tag.position = c(0, 1)
    )
  )

options(repr.plot.width = 20, repr.plot.height = 10, repr.plot.res = 800)
final_acf

ggsave(
  "acf_residuals.pdf",
  final_acf,
  device = cairo_pdf,
  width  = 12,
  height = 6,
  units  = "in",
  dpi    = 300
)


In [ ]:
#SUPPLEMENTARY TABLE 1: COLLINEARITY TEST
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
check_collinearity(first_pass_train, effects = "fixed")
check_collinearity(second_pass, effects = "fixed")

X <- train_data %>%
  select(adopted_lag, cost_proxy_scaled, spatial_influence_detrended, distance_to_waterway, distance_to_pipeline)

corrplot(cor(X, use = "complete.obs"), method = "color", tl.cex = 0.8)
cor(train_data$adopted_lag, train_data$cost_proxy_scaled) #Show high correlation between adopted lag and cost proxy

In [ ]:
#INLINE FIGURE ONLY: COST COMPETITIVENESS MEASURE OVER TIME
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
nrc_blue <- pal_npg("nrc")(10)[2]

theme_ne <- function(base_size = 20) {
  theme_minimal(base_size = base_size) +
    theme(
      panel.grid.major = element_blank(),
      panel.grid.minor = element_blank(),
      axis.line   = element_line(color = "black"),
      axis.ticks  = element_line(color = "black"),
      axis.text   = element_text(size = 14),
      axis.title  = element_text(size = 16),
      plot.title  = element_text(size = 18, hjust = 0.5),
      legend.position = "right",
      legend.title = element_text(size = 16),
      legend.text  = element_text(size = 14)
    )
}

median_iqr <- function(x) {
  data.frame(
    y    = median(x, na.rm = TRUE),
    ymin = quantile(x, 0.25, na.rm = TRUE),
    ymax = quantile(x, 0.75, na.rm = TRUE)
  )
}

median_df <- train_data %>%
  group_by(year) %>%
  summarise(median_val = median(cost_proxy_scaled, na.rm = TRUE), .groups = "drop")

years_all    <- sort(unique(train_data$year))
years_breaks <- years_all[years_all %% 10 == 0]

p_line_range_violin <- ggplot(train_data, aes(factor(year), cost_proxy_scaled)) +
  geom_violin(
    aes(colour = "Distribution shape"),
    fill = NA,
    linewidth = 0.4,
    trim = FALSE,
    show.legend = TRUE
  ) +
  stat_summary(
    fun.data = median_iqr,
    geom = "pointrange",
    aes(colour = "IQR (25–75%)"),
    size = 0.4,
    show.legend = TRUE
  ) +
  geom_line(
    data = median_df,
    aes(factor(year), median_val, group = 1),
    colour = "black",
    linewidth = 0.8,
    inherit.aes = FALSE
  ) +
  geom_point(
    data = median_df,
    aes(factor(year), median_val, colour = "Median"),
    size = 1.6,
    inherit.aes = FALSE,
    show.legend = TRUE
  ) +
  scale_colour_manual(
    name = "Statistics",
    values = c(
      "Distribution shape" = nrc_blue,
      "IQR (25–75%)"       = "black",
      "Median"             = "black"
    )
  ) +
  scale_x_discrete(breaks = years_breaks) +
  labs(
    title = "Yearly distribution of cost competitiveness proxy",
    x = "Year",
    y = "Cost competitiveness proxy (EUR/MWh simulation units)"
  ) +
  theme_ne() +
  theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust = 1))

options(repr.plot.width = 12, repr.plot.height = 6, repr.plot.res = 800)
p_line_range_violin

ggsave(
  "si-cost-competitiveness.pdf",
  p_line_range_violin,
  device = cairo_pdf,
  width = 12, height = 6, units = "in", dpi = 400
)
